In [156]:
import groq
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_groq import ChatGroq

In [ ]:
# setting env
from dotenv import load_dotenv
load_dotenv()

True

In [158]:
import os

In [ ]:
#to read the doc , using pypdf
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    doc = file_loader.load()
    return doc
     

In [160]:
docs = read_doc('document/')
docs

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [ ]:
#convert into chunks ...setting parameters
def chuck_data(docs,chunk_size = 500 ,chunk_overlap =100):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc1=text_splitter.split_documents(docs)
    return doc1

In [ ]:
#converted text to chunks
document = chuck_data(docs=docs)
document

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [ ]:
#embedding model config
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.environ["GOOGLE_API_KEY"]  
)
embeddings


GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001CB6056CFB0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000001CB601D4BF0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [ ]:
# into vectors from text 

vector= embeddings.embed_query("how you?")
vector

[0.04641544818878174,
 0.009740499779582024,
 -0.07725498080253601,
 -0.009471910074353218,
 0.07259868085384369,
 -0.0015576379373669624,
 0.006824759300798178,
 -0.03372160345315933,
 -7.295416435226798e-05,
 0.03963016718626022,
 -0.04446610063314438,
 0.017216337844729424,
 -0.029861290007829666,
 7.37392037990503e-05,
 -0.003300442360341549,
 -0.0452217273414135,
 0.001519463723525405,
 0.009967785328626633,
 0.031202904880046844,
 -0.013236057944595814,
 0.019108127802610397,
 0.04125399515032768,
 -0.014015428721904755,
 -0.00031718367245048285,
 0.026127850636839867,
 -0.03328917175531387,
 0.012886403128504753,
 -0.01854570209980011,
 -0.05242183804512024,
 0.0049650948494672775,
 -0.04603483900427818,
 0.029751170426607132,
 -0.029834995046257973,
 0.0006378395482897758,
 0.023728907108306885,
 -0.06624028831720352,
 0.008972051553428173,
 0.02470105141401291,
 -0.01989631913602352,
 0.00709198322147131,
 0.018053196370601654,
 -0.029912739992141724,
 -0.02841036021709442,
 0

In [272]:
from pinecone import Pinecone


In [ ]:
#setting env and key
pc = Pinecone(
    api_key="pcsk_5XNZiL_6jXjn6GhdNLDGGhRLyFe6wh5rWpf6V7P2JQsH6WRMZGM6r8KfdyCLYDwG5DrA2L",  # replace with your Pinecone API key
    environment="us-east-1"  # replace with your Pinecone environment
)

In [274]:
index_name = "my-embedding-index"
dimension = 768

In [ ]:
#i have created index of pinecone from notebook 
# cna also be create form pinecone website
#and setting all the requirements 

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        vector_type="dense",
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",          # Cloud provider
            region="us-east-1"    # Region
        )
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")


Index 'my-embedding-index' created.


In [ ]:

# Connect to the index ofpinecone console

index = pc.Index(index_name)
print("Index ready to use.")

Index ready to use.


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.utils import guard_import
from tenacity import retry, stop_after_attempt, wait_fixed

# this is a retry Mechanism – Built-in retry logic for pinecone operations
@retry(stop_after_attempt(3), wait_fixed(2))
def pinecone_upsert_safe(index, vectors):
    return index.upsert(vectors=vectors)


# Create vector store for ..storing all the embeddings in pinecone DB
index = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=index_name,
    
)



In [ ]:
#query part 
# using cosine similarity retriever serach from vecotr db

def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [286]:
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq

In [ ]:
#SETTIGN UP THE LLM MODEL , I HAVE llama-versatile from gro, one of the powerful model
llm=ChatGroq(model_name="llama-3.3-70b-versatile",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

C:\Users\Anumitha\AppData\Local\Temp\ipykernel_26668\716491743.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain=load_qa_chain(llm,chain_type="stuff")


In [ ]:
# SEARCHING ANSWER FROM THE VECTOR DB

def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
#ASKING QUERY 
our_query="what is was the view of the llms after the introduction of this transformers"
answer = retrieve_answers(our_query)
print(answer)

[Document(id='d0eda90e-aa6d-42e6-a585-0bd3b5379421', metadata={'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'book': 'Advances in Neural Information Processing Systems 30', 'created': '2017', 'creationdate': '', 'creator': 'PyPDF', 'date': '2017', 'description': 'Paper accepted and presented at the Neural Information Processing Systems Conference (http://nips.cc/)', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requi

In [ ]:
# CAN BE FURTHER USED AS QUIZ APPLICATION
our_query="Give a 5 MCQ questions from this"
answer = retrieve_answers(our_query)
print(answer)

[Document(id='c9f1996d-0f7d-4854-9ce7-788cb9cde7bf', metadata={'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'book': 'Advances in Neural Information Processing Systems 30', 'created': '2017', 'creationdate': '', 'creator': 'PyPDF', 'date': '2017', 'description': 'Paper accepted and presented at the Neural Information Processing Systems Conference (http://nips.cc/)', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requi